In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
#read the input
carDf= pd.read_csv('newCardata.csv')
carDate= pd.read_csv('dataWithDate.csv')
carLable= carDate['FraudFound']

#drop the Accident Date and Claim Date
carDate.drop(['FraudFound'],inplace=True,axis=1)
carDate.drop(['AccidentDate'],inplace=True,axis=1)
carDate.drop(['ClaimDate'],inplace=True,axis=1)

carDate.head()

print(type(carLable[0]))

<class 'numpy.int64'>


In [3]:

#select all the attributes of type object
car_object= carDate.select_dtypes(include=['object']).copy()

#drop the attributes of type object
carDate.drop(carDate.select_dtypes(['object']),inplace=True,axis=1)

carDate.head()
#print(type(carDate))

,RepNumber,Deductible,DriverRating,DayDiff
0,12,300,1,0
1,15,400,4,5
2,7,400,3,13
3,4,400,2,20
4,3,400,1,15


In [4]:
#one-hot representation preprocessing

car_copy = car_object.copy()
car_copy = pd.get_dummies(car_copy,columns=['Make'],prefix=['Make'])
car_copy = pd.get_dummies(car_copy,columns=['AccidentArea'],prefix=['AccidentArea'])
car_copy = pd.get_dummies(car_copy,columns=['Sex'],prefix=['Sex'])
car_copy = pd.get_dummies(car_copy,columns=['MaritalStatus'],prefix=['MartalStatus'])
car_copy = pd.get_dummies(car_copy,columns=['Fault'],prefix=['Fault'])
car_copy = pd.get_dummies(car_copy,columns=['PolicyType'],prefix=['PolicyType'])
car_copy = pd.get_dummies(car_copy,columns=['VehiclePrice'],prefix=['VehiclePrice'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Accident'],prefix=['Days:Policy-Accident'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Claim'],prefix=['Days:Policy-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['PastNumberOfClaims'],prefix=['PastNumberOfClaims'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfVehicle'],prefix=['AgeOfVehicle'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfPolicyHolder'],prefix=['AgeOfPolicyHolder'])
car_copy = pd.get_dummies(car_copy,columns=['PoliceReportFiled'],prefix=['PoliceReportFiled'])
car_copy = pd.get_dummies(car_copy,columns=['WitnessPresent'],prefix=['WitnessPresent'])
car_copy = pd.get_dummies(car_copy,columns=['AgentType'],prefix=['AgentType'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfSuppliments'],prefix=['NumberOfSuppliments'])
car_copy = pd.get_dummies(car_copy,columns=['AddressChange-Claim'],prefix=['AddressChange-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfCars'],prefix=['NumberOfCars'])

car_copy.head()
print(type(car_copy))

<class 'pandas.core.frame.DataFrame'>


In [5]:
car_copy['RepNumber']= carDate['RepNumber']
car_copy['Deductible']= carDate['Deductible']
car_copy['DriverRating']= carDate['DriverRating']
car_copy['DayDiff']= carDate['DayDiff']

car_copy.head(3)


,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNumber,Deductible,DriverRating,DayDiff
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,12,300,1,0
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,15,400,4,5
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,7,400,3,13


In [6]:
#normalization of feature to bring the value in the range [0,1]
minMaxScale = MinMaxScaler()
carDateNormalized = minMaxScale.fit_transform(car_copy)
#print(car_df_Scale)

#converting numpyarry to dataframe
carDateNormalized = pd.DataFrame(carDateNormalized)
carDateNormalized.head(3)
print(type(carDateNormalized))
print(carDateNormalized.iloc[0,96])

<class 'pandas.core.frame.DataFrame'>
0.0


/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [7]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])


j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1
    

In [8]:
print(nonFraudulent.shape)
print(fraudulent.shape)
nonFraudulent.head()

(14496, 97)
(923, 97)


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.733333,0.00,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.933333,0.25,1.000000,0.013587
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.400000,0.25,0.666667,0.035326
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.200000,0.25,0.333333,0.054348
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.133333,0.25,0.000000,0.040761


In [9]:
col= list(car_copy.columns.values)
nonFraudulent.columns= col
fraudulent.columns= col

In [10]:
nonFraudulent.head()
fraudulentLable.head()

,lable
28,1.0
52,1.0
53,1.0
94,1.0
96,1.0


In [11]:
#Note that for the train and test split, the lables and data have to be in numpy
nonFraudulentCarfeatures= nonFraudulent.values
nonFraudulentLable= nonFraudulentLable.values

fraudulentCarfeatures= fraudulent.values
fraudulentLable= fraudulentLable.values

print('non-Fraud: %d' %(nonFraudulentCarfeatures.shape[0]))
print('Fraud: %d' %(fraudulentCarfeatures.shape[0]))


non-Fraud: 14496
Fraud: 923


In [12]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [13]:
#split the dataset into two part train and test
"""
X_train,X_test,y_train,y_test = train_test_split(carArray,labels,random_state=3,test_size=0.25)
print('xtrain:',type(X_train))
print(X_train.shape)
"""

"\nX_train,X_test,y_train,y_test = train_test_split(carArray,labels,random_state=3,test_size=0.25)\nprint('xtrain:',type(X_train))\nprint(X_train.shape)\n"

In [13]:
#make the dataset for train and test loader
mytrainDataset= myDataset(nonFraudulentCarfeatures,nonFraudulentLable)
mytestDataset= myDataset(fraudulentCarfeatures,fraudulentLable)



In [14]:
feature,lable= mytrainDataset.__getitem__(0)
print("feature:",feature)
print("lable:",lable)
print(feature.size())

feature: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.7333, 0.0000, 0.0000, 0.0000])
lable: tensor([0.])
torch.Size([97])


In [15]:

trainLoader1= torch.utils.data.DataLoader(mytrainDataset,batch_size=1,shuffle=True,num_workers=0)
testLoader1= torch.utils.data.DataLoader(mytestDataset,batch_size=1,shuffle=True,num_workers=0)

trainLoader2= torch.utils.data.DataLoader(mytestDataset,batch_size=1,shuffle=True,num_workers=0)
testLoader2= torch.utils.data.DataLoader(mytrainDataset,batch_size=1,shuffle=True,num_workers=0)



In [16]:
#Network architecture for the base autoencoders
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder= nn.Sequential(
            nn.Linear(97,50),
            nn.ReLU(True),
            nn.Linear(50,25),
            nn.ReLU(True),nn.Linear(25,10),nn.ReLU(True),nn.Linear(10,5))
        self.decoder= nn.Sequential(
            nn.Linear(5,10),
            nn.ReLU(True),
            nn.Linear(10,25),
            nn.ReLU(True),
            nn.Linear(25,50),
            nn.ReLU(True),nn.Linear(50,97))
    
    def forward(self,x):
        x= self.encoder(x)
        x= self.decoder(x)
        return x;

In [17]:
model= autoencoder()
criterion= nn.MSELoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.001,momentum=0.9)

In [18]:
#train the model
for epoch in range(100):
    running_loss= 0.0
    for i, (feature,lable) in enumerate(trainLoader1):
        
        #gets the inputs
        inputs= torch.tensor(feature)
        lables= torch.tensor(lable)
        lables= lables.type(torch.LongTensor)
       
        #print(inputs.size())
        #print(inputs)
        
        # =====================forward====================
        output = model(inputs)
        #print(output)
        #loss = criterion(output,inputs).mean(dim=1)
        loss = criterion(output,inputs)
        
        #print("loss:",loss)
        #print("output:",output)
        #print("input:",inputs)
               
        
         # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # =======print the statistics
        running_loss += loss.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss / nonFraudulentCarfeatures.shape[0]))
      #running_loss = 0.0

        

[1] loss: 0.083
[2] loss: 0.072
[3] loss: 0.072
[4] loss: 0.072
[5] loss: 0.072
[6] loss: 0.072
[7] loss: 0.072
[8] loss: 0.072
[9] loss: 0.072
[10] loss: 0.072
[11] loss: 0.072
[12] loss: 0.071
[13] loss: 0.070
[14] loss: 0.069
[15] loss: 0.067
[16] loss: 0.067
[17] loss: 0.067
[18] loss: 0.067
[19] loss: 0.067
[20] loss: 0.067
[21] loss: 0.067
[22] loss: 0.066
[23] loss: 0.066
[24] loss: 0.063
[25] loss: 0.061
[26] loss: 0.060
[27] loss: 0.060
[28] loss: 0.060
[29] loss: 0.060
[30] loss: 0.059
[31] loss: 0.059
[32] loss: 0.059
[33] loss: 0.059
[34] loss: 0.058
[35] loss: 0.058
[36] loss: 0.058
[37] loss: 0.058
[38] loss: 0.058
[39] loss: 0.058
[40] loss: 0.058
[41] loss: 0.058
[42] loss: 0.058
[43] loss: 0.058
[44] loss: 0.058
[45] loss: 0.058
[46] loss: 0.057
[47] loss: 0.057
[48] loss: 0.057
[49] loss: 0.056
[50] loss: 0.056
[51] loss: 0.056
[52] loss: 0.056
[53] loss: 0.055
[54] loss: 0.055
[55] loss: 0.055
[56] loss: 0.055
[57] loss: 0.055
[58] loss: 0.054
[59] loss: 0.054
[60] l

In [19]:
 #test the model
def modelTest(Loader):
    predictedLable= []
    lossList= []
    for i,(feature,lable) in enumerate(Loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        loss= criterion(output,inputs)
        
        lossList.append(loss.item())
        
    return predictedLable,lossList
        

In [20]:
predictedLable,lossList= modelTest(testLoader1)
plus= predictedLable.count(1)
minus= predictedLable.count(0)
print('%d %d' %(plus,minus))

0 0


In [21]:
len(lossList)
lossList[1]

0.05060538649559021

In [22]:
#Network architecture for the base autoencoders
class autoencoder2(nn.Module):
    def __init__(self):
        super(autoencoder2,self).__init__()
        self.encoder2= nn.Sequential(
            nn.Linear(97,75),
            nn.LeakyReLU(True),
            nn.Linear(75,25),
            nn.LeakyReLU(True),nn.Linear(25,10))
        self.decoder2= nn.Sequential(
            nn.Linear(10,25),
            nn.LeakyReLU(True),
            nn.Linear(25,75),
            nn.LeakyReLU(True),
            nn.Linear(75,97))
    
    def forward(self,x):
        x= self.encoder2(x)
        x= self.decoder2(x)
        return x;

In [23]:
model2= autoencoder2()
criterion2= nn.MSELoss()
optimizer2= torch.optim.SGD(model2.parameters(), lr=0.001,momentum=0.9)

In [24]:
#train the model2
for epoch in range(50):
    running_loss2= 0.0
    for i, (feature,lable) in enumerate(trainLoader2):
        
        #gets the inputs
        inputs2= torch.tensor(feature)
        lables2= torch.tensor(lable)
        lables2= lables2.type(torch.LongTensor)
       
        #print(inputs.size())
        #print(inputs)
        
        # =====================forward====================
        output2= model2(inputs2)
        
        #print(output)
        #loss = criterion(output,inputs).mean(dim=1)
        loss2= criterion2(output2,inputs2)
        #print("loss2: ",loss2)
        '''
        print("loss:",loss)
        print("output:",output)
        print("input:",inputs)
        '''    
        
         # ===================backward====================
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
        
        # =======print the statistics
        running_loss2 += loss2.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss2 / fraudulentCarfeatures.shape[0]))
      #running_loss = 0.0

[1] loss: 0.141
[2] loss: 0.076
[3] loss: 0.069
[4] loss: 0.069
[5] loss: 0.068
[6] loss: 0.068
[7] loss: 0.068
[8] loss: 0.068
[9] loss: 0.068
[10] loss: 0.068
[11] loss: 0.068
[12] loss: 0.068
[13] loss: 0.068
[14] loss: 0.068
[15] loss: 0.067
[16] loss: 0.067
[17] loss: 0.067
[18] loss: 0.067
[19] loss: 0.066
[20] loss: 0.066
[21] loss: 0.066
[22] loss: 0.065
[23] loss: 0.065
[24] loss: 0.064
[25] loss: 0.063
[26] loss: 0.063
[27] loss: 0.062
[28] loss: 0.062
[29] loss: 0.061
[30] loss: 0.061
[31] loss: 0.060
[32] loss: 0.060
[33] loss: 0.059
[34] loss: 0.058
[35] loss: 0.058
[36] loss: 0.057
[37] loss: 0.057
[38] loss: 0.056
[39] loss: 0.056
[40] loss: 0.055
[41] loss: 0.055
[42] loss: 0.054
[43] loss: 0.054
[44] loss: 0.053
[45] loss: 0.053
[46] loss: 0.053
[47] loss: 0.052
[48] loss: 0.052
[49] loss: 0.052
[50] loss: 0.051


In [25]:
def modelTest2(Loader2):
    predictedLable2= []
    lossList2= []
    for i,(feature,lable) in enumerate(Loader2):
        inputs2= torch.tensor(feature)
        output2= model2(inputs2)
        loss2= criterion(output2,inputs2)
        
        lossList2.append(loss2.item())
        #print(loss)
        if(loss2>0.05):
            predictedLable2.append(0)
        else:
            predictedLable2.append(1)

    return predictedLable2,lossList2
        

In [26]:
predictedLable2,lossList2= modelTest2(testLoader1)
plus2= predictedLable2.count(1)
minus2= predictedLable2.count(0)
print('%d %d' %(plus2,minus2))

485 438


In [27]:
print("loss1:",lossList[1])
print("loss2:",lossList2[1])


loss1: 0.05060538649559021
loss2: 0.03439868241548538


In [28]:
prediction=[]
for k in range(len(lossList)):
    if(lossList[k]<lossList2[k]):
        prediction.append(0)
    else:
        prediction.append(1)

In [29]:
class1= prediction.count(1)
class0= prediction.count(0)
print('%d %d' %(class1,class0))

408 515


In [30]:
predictedLable3,lossList3= modelTest(testLoader2)
plus2= predictedLable3.count(1)
minus2= predictedLable3.count(0)
print('%d %d' %(plus,minus))

0 0


In [31]:
predictedLable4,lossList4= modelTest2(testLoader2)
plus2= predictedLable4.count(1)
minus2= predictedLable4.count(0)
print('%d %d' %(plus2,minus2))

4391 10105


In [32]:
prediction2=[]
for k in range(len(lossList3)):
    if(lossList3[k]<lossList4[k]):
        prediction2.append(0)
    else:
        prediction2.append(1)

In [33]:
class1= prediction2.count(1)
class0= prediction2.count(0)
print('%d %d' %(class1,class0))

3647 10849


In [36]:
#save the model
torch.save(model2.state_dict(),'../saved_model2')

In [39]:
#load the model
m= autoencoder2();
m.load_state_dict(torch.load('../saved_model2'))
m.eval()

autoencoder2(
  (encoder2): Sequential(
    (0): Linear(in_features=97, out_features=75, bias=True)
    (1): LeakyReLU(negative_slope=True)
    (2): Linear(in_features=75, out_features=25, bias=True)
    (3): LeakyReLU(negative_slope=True)
    (4): Linear(in_features=25, out_features=10, bias=True)
  )
  (decoder2): Sequential(
    (0): Linear(in_features=10, out_features=25, bias=True)
    (1): LeakyReLU(negative_slope=True)
    (2): Linear(in_features=25, out_features=75, bias=True)
    (3): LeakyReLU(negative_slope=True)
    (4): Linear(in_features=75, out_features=97, bias=True)
  )
)

In [40]:
def modelTest2(Loader2):
    predictedLable2= []
    lossList2= []
    for i,(feature,lable) in enumerate(Loader2):
        inputs2= torch.tensor(feature)
        output2= m(inputs2)
        loss2= criterion(output2,inputs2)
        
        lossList2.append(loss2.item())
        #print(loss)
        if(loss2>0.05):
            predictedLable2.append(0)
        else:
            predictedLable2.append(1)

    return predictedLable2,lossList2
        

In [41]:
p,l= modelTest2(testLoader1)
plus2= p.count(1)
minus2= p.count(0)
print('%d %d' %(plus2,minus2))

500 423


In [42]:
pred=[]
for k in range(len(lossList)):
    if(lossList[k]<l[k]):
        pred.append(0)
    else:
        pred.append(1)

In [43]:
plus2= pred.count(1)
minus2= pred.count(0)
print('%d %d' %(plus2,minus2))

704 219
